# <div style="background-color: lightgreen; padding: 10px;">Generation of paraphrases with GPT for Restaurant intents</div>

In [1]:
import os
import pathlib
import random

# let's just make sure we are at the root
os.chdir(pathlib.Path().absolute().parent)
current_directory = os.getcwd()
new_directory = f"{current_directory}/GPT_paraphrasing"
os.chdir(new_directory)
print(f"Current working directory: {os.getcwd()}")

Current working directory: /home/berro/Desktop/taxonomy_erro_model_execution/GPT_paraphrasing


In [2]:
import pandas as pd
from tqdm import tqdm
import time

import gpt_utility

## OpenAI credentials
Replace with your GPT API key and endpoint URL

In [3]:
# Replace with your personal GPT API key
API_KEY = "INSERT_YOUR_OPENAI_APT_KEY"

gpt_utility.define_credential(API_KEY)

### <div style="background-color: lightblue; padding: 10px;">Main generation function</div>

In [5]:
def run_generation(df, output_file):
    """
    Generate paraphrases for each utterance in the input dataframe.

    Parameters:
        df (pandas DataFrame): Input dataframe with the following columns: 'utterance', 'list_of_slots'.
        output_file (str): The filename for saving the paraphrases.

    Returns:
        None

    The function generates paraphrases for each row in the input dataframe, utilizing GPT-based paraphrasing.
    It crafts prompts, invokes GPT, and collects paraphrases, handling potential issues such as nan values
    and incomplete generations. The resulting paraphrases are saved to a CSV file for data persistence.

    Note:
        The function relies on external utility functions from the 'gpt_utility' module.

    Example:
        run_generation(df=my_dataframe, output_file="paraphrases_output.csv")
    """
    # Set the number of paraphrases to generate per prompt
    num_paraphrases = 10
    
    start_time = time.time()# Record the start time for performance evaluation
    
    result = list()# Initialize an empty list to store paraphrases

    # Iterate through each row in the dataframe
    for index, row in tqdm(df.iterrows(), total=df.shape[0], position=0, leave=True):
        # Step 1: Crafting Prompts for each seed_utterance
        prompt = gpt_utility.build_paraphrasing_prompt(row['utterance'], num_paraphrases)
        print(prompt)
        sys.exit()
    
        # Step 2: Invoke GPT with the built prompt
        # Handle issues such as nan values and incomplete generations
        # To prevent the occurrence of nan values, we set a maximum attempt limit.
        # This issue arises from GPT's token limit, particularly when the seed utterances are lengthy.
        # When generating more than 5 paraphrases, incomplete generation can happen.
        # For instance, if we aim to generate 10 paraphrases, we might end up with only 6
        attempts = 0
        max_attempts = 5
        collected_num_paraphrases = []

        # Initialize tqdm for the attempts loop
        with tqdm(total=max_attempts, desc=f"Attempt {attempts+1}", position=1, leave=False) as attempts_bar:
            while attempts < max_attempts:
                # Generate paraphrases
                paraphrases = gpt_utility.generate_paraphrases_with_retry(prompt, 10)

                # Check if GPT returned the specified number of paraphrases
                paraphrases = gpt_utility.checkgpt_generation_not_empty(paraphrases, num_paraphrases)

                # Check if the paraphrases list contains at least one nan value
                if any(pd.isna(p) for p in paraphrases):
                    # Collect non-nan candidates for further attempts
                    collected_num_paraphrases.extend([p for p in paraphrases if not pd.isna(p)])
                    collected_num_paraphrases = list(set(collected_num_paraphrases))

                    if len(collected_num_paraphrases) >= num_paraphrases:
                        # Randomly select the required number of paraphrases
                        paraphrases = random.sample(collected_num_paraphrases, num_paraphrases)
                        break
                    else:
                        attempts_bar.set_postfix(status=f"Retry: Some paraphrases contain nan values")
                        attempts_bar.update(1)
                else:
                    break

                attempts += 1  # Increment the attempts counter
                attempts_bar.update(1)
            
        # Step 3: Extract paraphrases and append to the result list
        for p in paraphrases:
            result.append([row['utterance'], p, row['list_of_slots']])
    
    print()
    print(f"Paraphrasing has been successfully accomplished.")
    print()
    
    # Create a dataframe from the result list
    column_names = ["utterance", "paraphrase", "list_of_entities"]
    df_result = pd.DataFrame(result, columns=column_names)
    
    # Save the dataframe to a CSV file for data persistence
    current_time = time.strftime("%Y%m%d-%H%M%S")
    path_to_output_file = os.path.join(os.getcwd(), output_file)
    df_result.to_csv(path_to_output_file, index=False)messages =[
		{
			"role": " system ",
			" content ": "You are a paraphrase generation model. Given an input sentence, generate diverse and coherent paraphrases while maintaining the original meaning."
		},
		{
			"role": "user",
			"content": 
				"Generate 10 paraphrases for the following sentence: Will it be  snowy  on  february the 14th, 2021  in  Port Armstrong ?.\n"\
				"Provide your response in a JSON format. Do not provide any additional information except the JSON. Your JSON response should respect this structure:"\
				"  {'paraphrases': ['paraphrase_1', 'paraphrase_2', Ellipsis, 'paraphrase_n']}\n\n"
		}
	]

### <div style="background-color: #FFBB5C; padding: 10px;">Run the generation for weather intent</div>

In [6]:
input_file = f"{os.getcwd()}/weather_dataset.csv"#test_data
df_weather = pd.read_csv(input_file)
print(f"Shape: {df_weather.shape}")
print(f"Columns: {df_weather.columns}")
df_weather.head()

Shape: (297, 2)
Columns: Index(['utterance', 'list_of_slots'], dtype='object')


,utterance,list_of_slots
0,How's the weather in Munchique National Natur...,['Munchique National Natural Park || geographi...
1,Tell me the weather forecast for France,['France || country']
2,Will there be wind in Hornitos DC ?,"['wind || condition_description', 'Hornitos ||..."
3,Is it warm here now ?,"['warm || condition_temperature', 'here || cur..."
4,what is the forecast for Roulo for foggy c...,"['Roulo || city', 'foggy || condition_descript..."


In [35]:
# Randomly samples 100 rows from the dataFrame.
sampled_weather_df = df_weather.sample(n=100, replace=False)
sampled_weather_df.reset_index(drop=True, inplace=True)

run_generation(sampled_weather_df,"GPT_weather")

Attempt 1:   0%|                                          | 0/5 [00:00<?, ?it/s]

Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

Retrying generation for current utterance:  17%|▎ | 1/6 [00:16<01:23, 16.74s/it]

Retrying generation for current utterance:  33%|▋ | 2/6 [00:31<01:01, 15.41s/it]

Retrying generation for current utterance:  50%|█ | 3/6 [00:46<00:45, 15.27s/it]

Retrying generation for current utterance:  67%|█▎| 4/6 [01:04<00:32, 16.38s/it]

Retrying generation for current utterance:  83%|█▋| 5/6 [01:18<00:15, 15.38s/it]

Retrying generation for current utterance: 100%|██| 6/6 [01:34<00:00, 15.91s/it]

                                                                                

Attempt 1: The generated list of paraphrases is empty. Retrying...




Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

Retrying generation for current utterance:  17%|▎ | 1/6 [00:18<01:30, 18.16s/it]

Retrying generation for current utterance:  33%|▋ | 2/6 [00:34<01:07, 16.84s/it]

                                                                                
Attempt 1:   0%| | 0/5 [02:21<?, ?it/s, status=Retry: Some paraphrases contain n
Attempt 1:  20%|▏| 1/5 [02:21<09:26, 141.59s/it, status=Retry: Some paraphrases 

Paraphrases generated successfully after 1 attempts.




Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

                                                                                
  1%|▍                                       | 1/100 [02:36<4:18:59, 156.97s/it]

...

Paraphrases generated successfully after 0 attempts.



Attempt 1:   0%|                                          | 0/5 [00:00<?, ?it/s]

Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

Retrying generation for current utterance:  17%|▎ | 1/6 [00:12<01:02, 12.53s/it]

Retrying generation for current utterance:  33%|▋ | 2/6 [00:33<01:09, 17.27s/it]

Retrying generation for current utterance:  50%|█ | 3/6 [00:58<01:03, 21.09s/it]

Retrying generation for current utterance:  67%|█▎| 4/6 [01:32<00:52, 26.05s/it]

                                                                                
Attempt 1:   0%| | 0/5 [02:01<?, ?it/s, status=Retry: Some paraphrases contain n
Attempt 1:  20%|▏| 1/5 [02:01<08:04, 121.06s/it, status=Retry: Some paraphrases 

Paraphrases generated successfully after 0 attempts.




Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

Retrying generation for current utterance:  17%|▎ | 1/6 [00:35<02:56, 35.32s/it]

                                                                                
100%|██████████████████████████████████████| 100/100 [4:51:44<00:00, 175.04s/it]

Paraphrases generated successfully after 0 attempts.

Paraphrasing has been successfully accomplished.



### <div style="background-color: lightgreen; padding: 10px;">Run the generation for book_restaurant</div>

In [37]:
input_file = f"{os.getcwd()}/book_restaurant.csv"#test_data
df_restaurant = pd.read_csv(input_file)
print(f"Shape: {df_restaurant.shape}")
print(f"Columns: {df_restaurant.columns}")
df_restaurant.head()

Shape: (300, 2)
Columns: Index(['utterance', 'list_of_slots'], dtype='object')


,utterance,list_of_slots
0,Book a reservation for seven people at a ba...,"['seven || party_size_number', 'bakery || rest..."
1,Book spot for three at Maid-Rite Sandwich S...,"['three || party_size_number', 'Maid-Rite Sand..."
2,I need a table for breakfast in MI at the ...,"['breakfast || timeRange', 'MI || state', 'piz..."
3,Book a restaurant reservation for me and my c...,"['me and my child || party_size_description', ..."
4,I want to book a highly rated churrascaria ...,"['highly rated || sort', 'churrascaria || rest..."


In [39]:
# Randomly samples 100 rows from the dataFrame.
sampled_restaurant_df = df_restaurant.sample(n=100, replace=False)
sampled_restaurant_df.reset_index(drop=True, inplace=True)

run_generation(sampled_restaurant_df,"GPT_restaurant.csv")

Attempt 1:   0%|                                          | 0/5 [00:00<?, ?it/s]

Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

                                                                                
Attempt 1:   0%| | 0/5 [00:12<?, ?it/s, status=Retry: Some paraphrases contain n
Attempt 1:  20%|▏| 1/5 [00:12<00:50, 12.68s/it, status=Retry: Some paraphrases c

Paraphrases generated successfully after 0 attempts.




Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

Retrying generation for current utterance:  17%|▎ | 1/6 [00:55<04:35, 55.20s/it]

Retrying generation for current utterance:  33%|▋ | 2/6 [02:49<05:59, 89.77s/it]

Retrying generation for current utterance:  50%|█ | 3/6 [03:28<03:20, 66.92s/it]

                                                                                
  1%|▍                                       | 1/100 [04:00<6:37:33, 240.94s/it]

Paraphrases generated successfully after 0 attempts.



Attempt 1:   0%|                                          | 0/5 [00:00<?, ?it/s]

Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

                                                                                
  2%|▊                                       | 2/100 [04:26<3:07:01, 114.51s/it]

...

Paraphrases generated successfully after 0 attempts.




Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

Retrying generation for current utterance:  17%|▎ | 1/6 [00:14<01:13, 14.65s/it]

Retrying generation for current utterance:  33%|▋ | 2/6 [00:37<01:17, 19.40s/it]

                                                                                
 99%|██████████████████████████████████████▌| 99/100 [6:19:55<02:02, 122.12s/it]

Paraphrases generated successfully after 0 attempts.



Attempt 1:   0%|                                          | 0/5 [00:00<?, ?it/s]

Retrying generation for current utterance:   0%|          | 0/6 [00:00<?, ?it/s]

                                                                                
100%|██████████████████████████████████████| 100/100 [6:20:33<00:00, 228.33s/it]

Paraphrases generated successfully after 0 attempts.

Paraphrasing has been successfully accomplished.

